In [39]:
include("StackyFan.jl")

coneRayDecomposition (generic function with 1 method)

In [40]:
#inputs are a stacky fan and a vector of booleans representing the distinguished structure.
function BerghA(F::StackyFan,D::Array{Int64,1})
    X=deepcopy(F)
    rayMatrix=convert(Array{Int64,2},Array(Polymake.common.primitive(X.fan.RAYS)))
    coneList=getCones(X.fan)
    dim=size(rayMatrix,2)
    l=size(rayMatrix,1)
    #check if the vector D has length equal to the number of rays in F
    if l != size(D,1)
        error("length of vector representing distinguished structure does not agree with number of rays in stacky fan.")
    end
    
    #A0
    while(true)
        rayMatrix=convert(Array{Int64,2},Array(Polymake.common.primitive(X.fan.RAYS)))
        coneList=getCones(X.fan)
        #A1
        #Note: cones in S are 1-indexed.
        S=filter(cone->distinguishedAndIntPoint(cone,rayMatrix,D),coneList)
        printn(rayMatrix)
        printn(S)
        if S==[]
            break
        end
        
        #A2 - find extremal cones
        Smax=extremalCones(S,rayMatrix,D)
        
        #A2 - find interior points in Smax
        intPoints=[]
        for cone in Smax
            C=coneConvert(cone,rayMatrix)
            coneIntPoints=interiorPoints(C)
            if coneIntPoints==nothing
                return C
            end
            for point in coneIntPoints
               push!(intPoints,(point,cone)) 
            end
        end
        
        #A2 - find stacky points (in terms of coefficients) derived from interior points
        P=Array{Int64,1}[]
        for (point,cone) in intPoints
            stackyPoint=coneRayDecomposition(cone,rayMatrix,point,X.scalars)
            push!(P,stackyPoint)
        end
        
        #A2 - find smallest element of P with respect to lex ordering.
        psi=minimalByLex(P)
        
        #A3 - perform root construction
        X=rootConstructionDistinguishedIndices(X,D,psi)
        
        #A3 - modify psi with respect to root construction
        for i in 1:length(psi)
            if D[i]==1 && psi[i]>0
                psi[i]=1
            end
        end
        
        while(count(x->x>0,psi)>1)
            #A4 - perform stacky star subdivision
            supportCone=findall(x->x!=0,psi)
            X=stackyBlowup(X,[x-1 for x in supportCone],findBarycenter(supportCone,X.fan))
            D=push!(D,1)
        
            #A4 - modify psi
            newpsi=zeros(Int64,length(psi)+1)
            for i in 1:length(psi)
                if psi[i]>1
                    newpsi[i]=psi[i]-1
                end
            end
            newpsi[end]=1
            psi=newpsi
        end
    end
    return X
end

BerghA (generic function with 1 method)

In [43]:
X=Polymake.fulton.NormalToricVariety(INPUT_RAYS=[1 2;1 0],INPUT_CONES=[[0,1]])

type: NormalToricVariety

INPUT_CONES
	{0 1}

INPUT_RAYS
	1 2
	1 0

In [45]:
F=addStackStructure(X,[3,4])

StackyFan(Polymake.BigObjectAllocated(Ptr{Nothing} @0x000000001a3db8c0), [3, 4], Dict("1,2" => 3,"1,0" => 4))

In [46]:
BerghA(F, [1, 1])

Polymake.Integer[1 2]Polymake.Integer[1 2][1 2; 1 0]
Any[]


StackyFan(Polymake.BigObjectAllocated(Ptr{Nothing} @0x000000001a3db8c0), [3, 4], Dict("1,2" => 3,"1,0" => 4))

In [286]:
P=interiorPoints(C)

Polymake.Integer[3 2 3; 1 2 0; 5 4 6]Polymake.Rational[1 0 0 0; 1 3 2 3; 1 1 2 0; 1 5 4 6; 1 4 4 3; 1 8 6 9; 1 6 6 6; 1 9 8 9]

type: Polytope<Rational>

POINTS
  1  0  0  0
  1  3  2  3
  1  1  2  0
  1  5  4  6
  1  4  4  3
  1  8  6  9
  1  6  6  6
  1  9  8  9

In [292]:
P.INTERIOR_LATTICE_POINTS

pm::Matrix<pm::Integer>
